In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hms-harmful-brain-activity-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F59093%2F7469972%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240419%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240419T031933Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0ce33d45c70d1474ca6421df40421451338501fc607fbaa39c6a8ffd2b43c76ec9201cbdf5fc2900cbd6d19de451185f3a44a36d28f1f17731499df2b7ab8a9e825598ae9dac4e42bb85fef7bb91be3dd4aef90198c5b21e4b785cefd897807be99ce0467098814572cf42d5920672fdd368f9fd84ca312a703c2d46fe9373342bba6cdc383d12c0e62de22faac2fb03a8c7787a9d0c4a82a18fc95025b72bca42a534782b812c242054447fcf55d11c6a4f4adb09873a50ce48fa638f3c75fe403090049394ed66aeb8e69380640e491e65a9893267572b498f83c6f67de6e73d1aeec2fde5396c4ef693f970a4b358b990c86f2fe975210bf8c5256655a9e6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 19762787900 bytes downloaded
Downloaded and uncompressed: hms-harmful-brain-activity-classification
Data source import complete.


## Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt, gc
from tqdm import tqdm

train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
print('Train shape', train.shape )
display( train.head() )

Train shape (106800, 15)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [ ]:
TARGETS = train.columns[-6:]

train_17k = train.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train_17k.columns = ['spec_id','min']

tmp = train.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train_17k['max'] = tmp

tmp = train.groupby('eeg_id')[['patient_id']].agg('first')
train_17k['patient_id'] = tmp

tmp = train.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train_17k[t] = tmp[t].values

y_data = train_17k[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train_17k[TARGETS] = y_data

tmp = train.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train_17k = train_17k.reset_index()
print('Train non-overlapp eeg_id shape:', train_17k.shape )
train_17k

Train non-overlapp eeg_id shape: (17089, 11)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1188113564,0.0,0.0,16610,0.0,0.000000,0.00,0.000000,0.500000,0.500000
17085,4293843368,1549502620,0.0,0.0,15065,0.0,0.000000,0.00,0.000000,0.500000,0.500000
17086,4294455489,2105480289,0.0,0.0,56,0.0,0.000000,0.00,0.000000,0.000000,1.000000
17087,4294858825,657299228,0.0,12.0,4312,0.0,0.000000,0.00,0.000000,0.066667,0.933333


In [ ]:
# sampling 40 % data only for doing experiments
train17k_40 = train_17k.sample(frac=0.4, random_state=1)
train17k_40

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
14848,3720367141,1086093682,0.0,12.0,54090,0.000000,0.000000,0.0,0.0,1.0,0.0
6395,1587069411,1355267267,0.0,0.0,15958,0.000000,0.000000,1.0,0.0,0.0,0.0
5169,1294960357,1336340733,660.0,660.0,37979,0.000000,0.000000,0.0,1.0,0.0,0.0
1941,496627552,2054909634,0.0,0.0,30631,0.000000,1.000000,0.0,0.0,0.0,0.0
15045,3772210564,1194143774,2226.0,2256.0,28330,0.277778,0.722222,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4825,1206216661,130827610,0.0,12.0,59915,0.000000,0.000000,0.0,0.0,1.0,0.0
16780,4210402294,412576653,0.0,8.0,54026,0.000000,0.000000,0.0,0.0,1.0,0.0
256,67705592,674570205,0.0,0.0,53977,0.000000,0.000000,0.0,0.0,0.0,1.0
6828,1687893617,148737241,0.0,0.0,32068,0.000000,0.000000,0.0,0.0,0.0,1.0


In [ ]:
# denoising function using wavelet transform
import pywt

def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='db8', level=1):
    #x = x.iloc[:, 1:-7]
    ret = {key:[] for key in x.columns}

    for pos in x.columns:
        coeff = pywt.wavedec(x[pos], wavelet, mode="per")
        sigma = (1/0.6745) * maddest(coeff[-level])

        uthresh = sigma * np.sqrt(2*np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

        ret[pos]=pywt.waverec(coeff, wavelet, mode='per')

    return pd.DataFrame(ret)

# usage

#denoised_eeg_data

In [ ]:
# DWT
from pywt import wavedec

def wavelet_decompose_channels(data, level, output=False):
  # take every x number of points using numpy's slicing (start:stop:step)
    data = data[0::2]

    data.columns.name='channel'

    # transpose the data
    data_t = data.transpose()

    # get the wavelet coefficients at each level in a list
    coeffs_list = wavedec(data_t.values, wavelet='db4', level=level)
    #print(len(coeffs_list))

    # make a list of the component names (later column rows)
    nums = list(range(1,level+1))
    names=[]
    for num in nums:
        names.append('D' + str(num))
    names.append('A' + str(nums[-1]))

  # reverse the names so it counts down
    names = names[::-1]
    #print(names)

    i = 0
    wavelets = pd.DataFrame()
    for i in range(1, len(coeffs_list)):
    #for i, array in enumerate(coeffs_list):
        #print(i)
        array = coeffs_list[i]
        # turn into a dataframe and transpose
        level_df = pd.DataFrame(array)
        level_df.index = data.columns
        level_df['level'] = names[i]
        level_df= level_df.set_index('level', append=True)
        level_df=level_df.T
        # add the next levels df to another column
        wavelets = pd.concat([wavelets,level_df], axis=1, sort=True)

    # sort values along the channels
    wavelets = wavelets.sort_values(['channel', 'level'], axis=1)

  # remove the AN levels
  #regex = re.compile('D')
  #bad_items = [x for x in list(wavelets.columns.levels[1]) if not regex.match(x)]
  #decom_wavelets = wavelets.drop(bad_items, axis=1, level = 'level')

  #decom_wavelets.index.name='sample'

  #if output:
  #  display(decom_wavelets.head())

    wavelets_cleaned = wavelets.dropna()

    return wavelets_cleaned

# usage
#dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)
#dwt_wavelets

In [ ]:
from scipy.stats import entropy

def MAV(data):
    # Initialize an empty DataFrame to store the means for each channel
    means = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        means[channel] = data[channel].abs().mean(axis=1)

    means.columns = [f"{col}_DT_MAV" for col in means.columns]

    return means


def MAVP(data):
    # Initialize an empty DataFrame to store the means for each channel
    means_abs = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        means_abs[channel] = (data[channel]**2).mean(axis = 1)

    means_abs.columns = [f"{col}_DT_MAVP" for col in means_abs.columns]

    return means_abs



def std_val(data):
    # Initialize an empty DataFrame to store the means for each channel
    std_vals = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        std_vals[channel] = data[channel].std(axis = 1)

    std_vals.columns = [f"{col}_DT_STD" for col in std_vals.columns]

    return std_vals


def var_val(data):
    # Initialize an empty DataFrame to store the means for each channel
    var = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        var[channel] = data[channel].var(axis = 1)

    var.columns = [f"{col}_DT_VAR" for col in var.columns]

    return var


def ratio_channels(epoch_data):
    # Initialize an empty DataFrame to store the ratio values for each pair of channels
    ratio_data = pd.DataFrame(index=epoch_data.index)

    # Iterate over each pair of adjacent channels
    for i in range(len(epoch_data.columns) - 1):
        channel1 = epoch_data.columns[i]
        channel2 = epoch_data.columns[i + 1]

        # Calculate the ratio between the values of the two channels
        ratio_data[f"{channel1}-{channel2}_Ratio"] = epoch_data[channel1] / epoch_data[channel2]

    return ratio_data



def shannon_entropy(data):
    # Initialize an empty DataFrame to store the entropy values for each channel
    entropy_df = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the entropy across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
        # Calculate the entropy for the current channel
        entropy_values = data[channel].apply(lambda x: entropy(x.abs(), base=2), axis=1)
        entropy_df[channel + '_entropy'] = entropy_values

    return entropy_df

## Experiment 1: Only DWT features

Code for DWT above

In [ ]:
def feature_exp_1(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)


    print(f"features done {eeg_id}")

    return dwt_wavelets


## Experiment a: Only CWT features

In [ ]:
def feature_exp_a(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)


    print(f"features done {eeg_id}")

    return dwt_wavelets


## Experiment 2: DWT with Statistical features

In [ ]:
def feature_exp_2(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)

    eegid_data_features = eeg_feature(dwt_wavelets)


    print(f"features done {eeg_id}")

    return eegid_data_features


## Experiment 3: DWT with Entropy features only

In [ ]:
def feature_exp_3(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)

    eegid_data_features = eeg_feature(dwt_wavelets)


    print(f"features done {eeg_id}")

    return eegid_data_features

## Experiment 4: DWT with all features without filtering

In [ ]:
def feature_exp_4(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)

    eegid_data_features = eeg_feature(dwt_wavelets)


    print(f"features done {eeg_id}")

    return eegid_data_features

## Experiment 5: DWT with all features with filtering

In [ ]:
def feature_exp_5(parquet_path, eegid, display=False):


    eeg = pd.read_parquet(parquet_path)

    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)

    eegid_data_features = eeg_feature(dwt_wavelets)


    print(f"features done {eeg_id}")

    return eegid_data_features

In [ ]:
def eeg_feature(data):

  wavelet_mean = MAV(data)
  wavelet_meanabs = MAVP(data)
  wavelet_std = std_val(data)
  wavelet_var = var_val(data)
  wavelet_ratio = ratio_channels(data)
  wavelet_entropy = shannon_entropy(data)

  wavelet_statistics = pd.concat([data, wavelet_mean, wavelet_meanabs, wavelet_std, wavelet_var, wavelet_ratio, wavelet_entropy], axis=1)


  wavelet_statistics.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in wavelet_statistics.columns]


  features_to_drop = ['C4_D1', 'F3_D1', 'F4_D1', 'F7_D1', 'F8_D1', 'Fp2_D1', 'Fz_D1', 'O2_D1', 'T3_D1', 'T4_D1', 'T5_D1', 'T6_D1', "('Fp2', 'D1')-('Fp2', 'D2')_Ratio", 'C3_entropy', 'C4_entropy', 'Cz_entropy', 'EKG_entropy', 'F3_entropy', 'F4_entropy', 'F7_entropy', 'F8_entropy', 'Fp1_entropy', 'Fp2_entropy', 'Fz_entropy', 'O1_entropy', 'O2_entropy', 'P3_entropy', 'P4_entropy', 'Pz_entropy', 'T3_entropy', 'T4_entropy', 'T5_entropy', 'T6_entropy']

  # Drop the specified features from the DataFrame
  wavelet_statistics.drop(columns=features_to_drop, inplace=True)

  return wavelet_statistics



In [ ]:
    #wavelet_statistics = pd.concat([data, wavelet_mean, wavelet_meanabs, wavelet_std, wavelet_var, wavelet_ratio], axis=1)
    #wavelet_statistics = pd.concat([data, wavelet_entropy], axis=1)

In [ ]:
# using sample data for trial
import random

ids = train17k_40.eeg_id.unique()

#n = 10
#ids1 = random.sample(list(ids), n)
#ids1

In [ ]:
def feature_eng_eeg(parquet_path, eegid, display=False):

#     # extract 10 seconds
#     eegid_data = pd.read_parquet(parquet_path)
#     all_segments = []
#     segment_length = 10  # 10-second EEG segments

#     # Filter training data for the current EEG ID
#     train_eegid = train[train['eeg_id'] == eeg_id]

#     offset_values_list = train_eegid['eeg_label_offset_seconds'].tolist()
#     print("Number of offset subsamples for EEG ID", eeg_id, ":", len(offset_values_list))

#     # Extract 10-second EEG segments along with target labels
#     for offset in offset_values_list:
#         start_index = int(offset) * 200
#         end_index = start_index + (segment_length * 200)
#         # Extract 10-second segment centered around the offset
#         middle_index = (start_index + end_index) // 2
#         segment_start = middle_index - (segment_length // 2 * 200)
#         segment_end = middle_index + (segment_length // 2 * 200)
#         # Extract EEG segment
#         eeg_segment = eegid_data.iloc[segment_start:segment_end].reset_index(drop=True)
#         all_segments.append(eeg_segment)
#     # Concatenate all segments into a single DataFrame
#     full_eeg_segments = pd.concat(all_segments, ignore_index=True)

    GET_ROW = 0

    row = train.iloc[GET_ROW]

    eeg = pd.read_parquet(parquet_path)
    #eeg_offset = int( row.eeg_label_offset_seconds ) # taking first offset only
    #full_eeg_segments = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]


    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8')

    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)

    # getting all features
    eegid_data_features = eeg_feature(dwt_wavelets)

    print(f"features done {eeg_id}")

    return eegid_data_features


In [ ]:
%%time
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'

EEG_IDS = ids
#EEG_IDS = ids1 # use this for sample data
all_eegs = {}
# for i,eeg_id in tqdm(enumerate(EEG_IDS)): # use this when in kaggle notebook
for i,eeg_id in enumerate(EEG_IDS):         # use this when saving output
    if (i%100==0)&(i!=0):
        print(i,', ',end='')


    # CREATE FEATURES FROM EEG PARQUET
    all_features_data = feature_exp_5(f'{PATH}{eeg_id}.parquet',eeg_id)
    # Remove rows with NaN values in feature data
    clean_features_data = all_features_data.dropna()

    if clean_features_data.empty:
        continue


    # GET TARGET LABELS FOR SPECIFIC EEF_ID
    train17k_40_eegid = train17k_40[train17k_40['eeg_id'] == eeg_id]
    target_labels = train17k_40_eegid.iloc[0][['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]

    # SAVE
    all_eegs[eeg_id] = (clean_features_data, target_labels)

    #if (i%1000==0)&(i!=0):
        #np.save(f'/kaggle/working/full_dataset{i}',all_eegs)
        #all_eegs = {}
        #print(i)


# # SAVE EEG SPECTROGRAM DICTIONARY
#np.save('/kaggle/working/full_dataset',all_eegs)

features done 3720367141
features done 1587069411
features done 1294960357
features done 496627552
features done 3772210564
features done 3128310317
features done 1168450679
features done 2265908441
features done 2901354063
features done 2270720155
features done 3399913366
features done 1232493771
features done 4060610136
features done 1604920026
features done 2063548848
features done 3834911964
features done 4065695514
features done 1347244748
features done 653601486
features done 2378686412
features done 406002683
features done 4038523251
features done 3378017081
features done 3103229069
features done 2108012317
features done 888030557
features done 456206864
features done 3763626214
features done 1385274250
features done 2130289282
features done 1965091244
features done 2631587463
features done 1573649480
features done 513227549
features done 3559891043
features done 1507393376
features done 1931533444
features done 3958123849
features done 993278641
features done 2881327473
feature

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


features done 1704960633
features done 2746968820
features done 745051336
features done 472904921
features done 3114745346
features done 1365048776
features done 354214183
features done 4118386827
features done 3013296315
features done 180905285
features done 3592820898
features done 2326735177
features done 2187560710
features done 299571149
features done 4018092903
features done 3106718920
features done 1433070974
features done 4161360477
features done 1732840935
features done 500990288
features done 813393221
features done 409444899
features done 89270417
features done 2721558761
features done 3782126576


/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


features done 4246240734
features done 3678337130
features done 1703105430
features done 477692850
features done 3094312124
features done 2071325747
features done 1897447067
features done 314897180
features done 334994359
features done 2036913942
features done 1524014528
features done 1962605623
features done 871834967
features done 4268847281
features done 1639774149
features done 3417915558
features done 2331147921
features done 850390848
features done 3452712208
features done 2388143719
200 , features done 3503391180
features done 3205174172
features done 331433920
features done 861804910
features done 2395992651
features done 1312905848
features done 3125655086
features done 493728415
features done 2328525878
features done 1932198093
features done 273637412
features done 17295749
features done 262635085
features done 1197932931
features done 3614914702
features done 1544035805
features done 1649977763
features done 1489958697
features done 2326371532
features done 203062137
feature

...

features done 1639794967
features done 43566640
features done 1759635757
features done 682038485
features done 1975646969
6300 , features done 2021338180
features done 4201830302
features done 1737483993
features done 1200683157
features done 667501054
features done 1062096035
features done 3079910575
features done 938923616
features done 2097822947
features done 3927828931
features done 2294065685
features done 3210633494
features done 400497237
features done 1031588198
features done 142529460
features done 1496963406
features done 2546952529
features done 2618856908
features done 3268705056
features done 98229238
features done 3604640101
features done 2687138980
features done 3985564312
features done 3782964497
features done 546754698
features done 2975420933
features done 3123791750
features done 2091277497
features done 3973731145
features done 2188217958
features done 510099548
features done 1802359359
features done 2613724527
features done 3379211428
features done 1022751153
feat

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


features done 3090694153
features done 2466764682
features done 4233815620
features done 1774403974
features done 142311321
features done 1075442225
features done 1744477250
features done 2662321842
features done 3519520666
features done 3597856220
features done 1641954518
features done 1256215657
features done 3190213597
features done 2294549048
features done 1492067494
features done 2411473850
features done 2559567335
features done 2023711129
features done 1877024480
features done 725185416
features done 1295891314
features done 3694266560
features done 2849338466
features done 536110625
features done 2047180059
features done 2648368617
features done 2311167068
features done 3978200112
features done 2894814743
features done 1172394549
features done 3744986076
features done 3724585250
features done 68849904
features done 3419431591
features done 2006486254
features done 3252014112
features done 1389727525
features done 3019144222
features done 3550217953
features done 2565742513
6400 

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


features done 876238217
features done 2962427242
features done 919409388
features done 3226913737
features done 3737225419
features done 1689206386
features done 1114003909
features done 2974663288
features done 3851517872
features done 1380800623
features done 4133663113
features done 3576578769
features done 913854185
features done 3575626106
features done 1381764246
features done 859827913
features done 3944383666
features done 1306925562
features done 2747666948
features done 2596517481
features done 936670859
features done 1748155103
features done 846573096
features done 3758950107
features done 2358826065
features done 937201320
features done 738217809
features done 3406318890
features done 1488374537
features done 2183796489
features done 3016732497
features done 548310032
features done 3496508985
features done 4150005169
features done 757967895
features done 3402647172
features done 1370616538
features done 700721907
features done 3538871931
features done 3776760596
features do

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

# Specify the file name
filename = '/content/drive/MyDrive/Colab Notebooks/my_dict5.pkl'

# Writing to a pickle file
with open(filename, 'wb') as f:
    pickle.dump(all_eegs, f)

In [ ]:
import pickle

# Adjust the path below to the location of your file in Drive
file_path = '/content/drive/My Drive/path_to_your_file/yourfile.pkl'

with open(file_path, 'rb') as f:
    data = pickle.load(f)

# Use the loaded data
print(data)
